In [55]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

data set link : https://www.kaggle.com/datasets/jangedoo/utkface-new

In [56]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [58]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [59]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [60]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [61]:
len(age)

23708

In [62]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [63]:
df.shape

(23708, 3)

In [64]:
df.head()

,age,gender,img
0,3,0,3_0_0_20170110212754904.jpg.chip.jpg
1,54,1,54_1_0_20170104212730503.jpg.chip.jpg
2,26,1,26_1_2_20170116180753524.jpg.chip.jpg
3,24,0,24_0_0_20170117145308498.jpg.chip.jpg
4,33,0,33_0_4_20170117202448894.jpg.chip.jpg


In [65]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [66]:
train_df.shape

(20000, 3)

In [67]:
test_df.shape

(3708, 3)

In [68]:
train_df.head()

,age,gender,img
15426,72,1,72_1_1_20170120225033578.jpg.chip.jpg
14084,85,1,85_1_0_20170110183522560.jpg.chip.jpg
22319,31,1,31_1_1_20170116234434494.jpg.chip.jpg
19518,61,1,61_1_3_20170119211810610.jpg.chip.jpg
2255,55,0,55_0_0_20170120222221343.jpg.chip.jpg


In [69]:
test_df.head()

,age,gender,img
9293,48,1,48_1_1_20170109220301068.jpg.chip.jpg
7515,25,0,25_0_1_20170113184508496.jpg.chip.jpg
53,47,0,47_0_4_20170104002129509.jpg.chip.jpg
6395,40,0,40_0_0_20170104170254385.jpg.chip.jpg
21250,28,1,28_1_0_20170117140041948.jpg.chip.jpg


In [70]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [81]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col= ['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [72]:
for i in next(train_generator):
  print (len(i))

32
2


In [73]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [74]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [75]:
resnet.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 200, 200, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 206, 206, 3)    │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 100, 100, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 100, 100, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 100, 100, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 102, 102, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 50, 50, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 50, 50, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 50, 50, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 50, 50, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 50, 50, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 50, 50, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 50, 50, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 50, 50, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 50, 50, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [76]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [77]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [78]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [79]:
# model.fit(train_generator, batch_size=32, steps_per_epoch=len(train_df) // 32, epochs=10, validation_data=test_generator)

TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

# got error in above code so using this code i do not understand what was that but solves the error

In [82]:
import tensorflow as tf

def generator(df, folder_path, datagen, image_size):
    for _, row in df.iterrows():
        img_path = os.path.join(folder_path, row['img'])
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = datagen.random_transform(img)
        img = datagen.standardize(img)

        age_label = row['age']
        gender_label = row['gender']

        yield img, (age_label, gender_label)

# Define output signature for the dataset
output_signature = (
    tf.TensorSpec(shape=(200, 200, 3), dtype=tf.float32),  # Image tensor
    (
        tf.TensorSpec(shape=(), dtype=tf.float32),  # Age (scalar)
        tf.TensorSpec(shape=(), dtype=tf.float32)   # Gender (scalar)
    )
)

# Create train and test datasets
train_dataset = tf.data.Dataset.from_generator(
    lambda: generator(train_df, folder_path, train_datagen, (200, 200)),
    output_signature=output_signature
)

test_dataset = tf.data.Dataset.from_generator(
    lambda: generator(test_df, folder_path, test_datagen, (200, 200)),
    output_signature=output_signature
)

# Set up batching for train and test datasets
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [84]:
# Fix: Ensure loss weights are float values
model.compile(optimizer='adam',
              loss={'age': 'mae', 'gender': 'binary_crossentropy'},
              metrics={'age': 'mae', 'gender': 'accuracy'},
              loss_weights={'age': 1.0, 'gender': 99.0})

In [85]:
model.fit(train_dataset,
          steps_per_epoch=len(train_df) // 32,
          epochs=10,
          validation_data=test_dataset,
          validation_steps=len(test_df) // 32)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 255s 382ms/step - age_mae: 15.9998 - gender_accuracy: 0.5131 - loss: 194.3913 - val_age_mae: 14.8332 - val_gender_accuracy: 0.5228 - val_loss: 83.3610
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - age_mae: 0.0000e+00 - gender_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_age_mae: 15.5311 - val_gender_accuracy: 0.5000 - val_loss: 84.2025
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 229s 366ms/step - age_mae: 15.0892 - gender_accuracy: 0.5154 - loss: 83.7176 - val_age_mae: 14.6060 - val_gender_accuracy: 0.5228 - val_loss: 83.1358
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - age_mae: 0.0000e+00 - gender_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_age_mae: 15.3250 - val_gender_accuracy: 0.5000 - val_loss: 83.9925
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 230s 369ms/step - age_mae: 15.0086 - gender_accuracy: 0.5155 - loss: 83.6878 - val_age_mae: 14.5120 - val_gender_accuracy: 0.5228 - val_loss: 83.0459
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - age_mae: 0.0000e+00 - gender_accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_age_mae: 15.2295 - val_gender_accuracy: 0.5000 - val_loss: 83.8897
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 230